In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from pettingzoo.mpe import simple_tag_v2
import os

# Initialize the environment
env = simple_tag_v2.parallel_env(render_mode=None, num_adversaries=4, num_good=1, num_obstacles=2)
env.reset()

# Parameters
num_class_a = 3
num_class_b = 1
num_adversaries = 4
num_agents = 1  # Only one agent being chased by adversaries
num_obstacles = 2
adversary_agents = [agent for agent in env.agents if 'adversary' in agent]
good_agents = [agent for agent in env.agents if 'agent' in agent]

print("Adversary Agents:", adversary_agents)
print("Good Agents:", good_agents)


Adversary Agents: ['adversary_0', 'adversary_1', 'adversary_2', 'adversary_3']
Good Agents: ['agent_0']


### Define Embedding and CGN layer

In [5]:
# Define linear embedding layer
class LinearEmbedding(nn.Module):
    def __init__(self, input_dim, embed_dim):
        super(LinearEmbedding, self).__init__()
        self.linear = nn.Linear(input_dim, embed_dim)
    
    def forward(self, x):
        return self.linear(x)

# Define GCN layer
class GCNLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GCNLayer, self).__init__()
        self.conv1 = pyg_nn.GCNConv(input_dim, output_dim)
    
    def forward(self, x, edge_index):
        return self.conv1(x, edge_index)

# Parameters for embedding and GCN
communication_range = 1.5  # Define the communication range
embedding_dim = 8          # Dimension for linear embedding
gcn_output_dim = 16        # Dimension of GCN output

# Initialize embedding and GCN layers
embedding_layer = LinearEmbedding(input_dim=5, embed_dim=embedding_dim)  # input_dim includes the class identifier
gcn_layer = GCNLayer(input_dim=embedding_dim, output_dim=gcn_output_dim)


### Observation Wrapper

In [6]:
def adversary_observation_wrapper(observations, num_class_A, num_class_B, adversary_agents, num_adversaries, num_agents, num_obstacles):
    assert num_class_A + num_class_B == num_adversaries, "Number of agents assigned to Class A and Class B must match the total number of adversaries."
    
    class_a_agents = adversary_agents[:num_class_A]
    class_b_agents = adversary_agents[num_class_A:num_class_A + num_class_B]
    
    updated_observations = {}
    adversary_positions = {}  # Store adversary positions for communication
    
    # Step 1: Add agent class identifier into observation
    for agent, obs in observations.items():
        # If the agent is an adversary (either Class A or Class B)
        if agent in class_a_agents or agent in class_b_agents:
            agent_class = 0 if agent in class_a_agents else 1  # Class A: 0, Class B: 1
            updated_obs = np.concatenate([obs, [agent_class]])  # Add class identifier
        else:
            updated_obs = obs  # Non-adversary agents keep their observation
        updated_observations[agent] = updated_obs
    
    # Step 2: Gather positions for communication
    for agent in adversary_agents:
        position = updated_observations[agent][2:4]  # Assume position is at index 2:4
        adversary_positions[agent] = position
    
    # Step 3: Apply linear embedding and handle communication within range
    embedded_information = {}
    for agent, obs in updated_observations.items():
        if agent in adversary_agents:
            # Convert observation to torch tensor
            obs_tensor = torch.tensor(obs[:5], dtype=torch.float32)  # Include the class identifier
            embedded_obs = embedding_layer(obs_tensor)  # Apply embedding to observation
            embedded_information[agent] = embedded_obs
    
    # Step 4: Communication and gather data for GCN input
    node_features = []
    edge_index = []
    num_agents_in_graph = len(adversary_agents)
    
    agent_to_idx = {agent: idx for idx, agent in enumerate(adversary_agents)}
    
    for agent in adversary_agents:
        own_position = adversary_positions[agent]
        node_features.append(embedded_information[agent])
        
        for other_agent in adversary_agents:
            if agent != other_agent:
                other_position = adversary_positions[other_agent]
                distance = np.linalg.norm(own_position - other_position)
                
                if distance <= communication_range:  # Check if within communication range
                    edge_index.append([agent_to_idx[agent], agent_to_idx[other_agent]])  # Add edge

    # Convert to torch tensors
    if edge_index:
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    else:
        edge_index = torch.empty((2, 0), dtype=torch.long)
    
    node_features = torch.stack(node_features)
    
    # Step 5: Pass the gathered information into a GCN
    gcn_output = gcn_layer(node_features, edge_index)
    
    # Step 6: Concatenate GCN output with agent's own observation
    final_inputs = {}
    for agent_idx, agent in enumerate(adversary_agents):
        own_obs = torch.tensor(updated_observations[agent], dtype=torch.float32)
        final_input = torch.cat([own_obs, gcn_output[agent_idx]])
        final_inputs[agent] = final_input  # This will be used as input to the Actor network
    
    return updated_observations, final_inputs  # Return the updated observations and inputs for Actor network


### Initialize the Environment and Run a Step

In [7]:
# Initialize environment
env.reset()

# Sample action spaces for all agents
actions = {agent: env.action_space(agent).sample() for agent in env.agents}

# Step through the environment
observations, rewards, terminations, truncations, infos = env.step(actions)

# Apply the observation wrapper for adversaries
observations, final_inputs = adversary_observation_wrapper(
    observations, num_class_a, num_class_b, adversary_agents, num_adversaries, num_agents, num_obstacles)

print("Updated Observations:")
for agent, obs in observations.items():
    print(f"{agent}: {obs}")

print("\nFinal Inputs for Actor Network:")
for agent, inp in final_inputs.items():
    print(f"{agent}: {inp}")

Updated Observations:
adversary_0: [ 0.30000001 -0.         -0.92546493 -0.7244485   0.55810302  0.99061686
  1.54220319  1.34548914  0.54744869  0.61472976  0.81065941 -0.13042034
  0.62077808  0.22459681  1.27130783  0.2539244   0.          0.
  0.        ]
adversary_1: [-0.30000001 -0.         -0.3780162  -0.10971878  0.01065429  0.3758871
  0.99475449  0.73075944 -0.54744869 -0.61472976  0.26321071 -0.74515009
  0.07332934 -0.39013293  0.72385913 -0.36080533  0.          0.
  0.        ]
adversary_2: [ 0.         -0.         -0.11480551 -0.85486889 -0.25255641  1.12103724
  0.73154378  1.47590959 -0.81065941  0.13042034 -0.26321071  0.74515009
 -0.18988135  0.35501716  0.46064845  0.38434476  0.          0.
  0.        ]
adversary_3: [ 0.30000001 -0.         -0.30468687 -0.4998517  -0.06267506  0.76602
  0.9214251   1.12089241 -0.62077808 -0.22459681 -0.07332934  0.39013293
  0.18988135 -0.35501716  0.6505298   0.0293276   0.          0.
  1.        ]
agent_0: [ 0.          0.     

### Actor Network


In [8]:
class ActorNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ActorNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, output_dim)  # Output dimension should match the action space
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize Actor networks for adversaries
actor_networks = {}
for agent in adversary_agents:
    input_dim = len(env.observation_space(agent).low) + 1 + gcn_output_dim  # Observation length plus class id plus GCN output
    output_dim = env.action_space(agent).n  # Assuming discrete action space
    actor_networks[agent] = ActorNetwork(input_dim, output_dim)


### Training Loop

In [9]:
# Training parameters
num_episodes = 50  # Total number of episodes to run
print_interval = 10  # Print rewards every 10 episodes

# Initialize reward tracking
episode_rewards = []
cumulative_reward = 0

# Optimizers for Actor networks and GCN (assuming we are training them)
learning_rate = 0.001
actor_optimizers = {agent: torch.optim.Adam(actor_networks[agent].parameters(), lr=learning_rate) for agent in adversary_agents}
gcn_optimizer = torch.optim.Adam(gcn_layer.parameters(), lr=learning_rate)

# Loss function (placeholder, you need to define based on your RL algorithm)
loss_fn = nn.MSELoss()

# Training loop
for episode in range(1, num_episodes + 1):
    observations = env.reset()
    done = False
    cumulative_reward = 0  # Reset cumulative reward for the episode
    
    while not done:
        # Apply the observation wrapper for adversaries
        observations, final_inputs = adversary_observation_wrapper(
            observations, num_class_a, num_class_b, adversary_agents, num_adversaries, num_agents, num_obstacles)
        
        actions = {}
        for agent in env.agents:
            if agent in adversary_agents:
                # Get the input for the Actor network
                actor_input = final_inputs[agent]
                # Get action probabilities (assuming discrete action space)
                action_probs = actor_networks[agent](actor_input)
                # Sample an action (for simplicity, we take the action with the highest probability)
                action = torch.argmax(action_probs).item()
                actions[agent] = action
            else:
                # For non-adversary agents, sample random actions
                actions[agent] = env.action_space(agent).sample()
        
        # Step the environment
        next_observations, rewards, terminations, truncations, infos = env.step(actions)
        
        # Update cumulative reward
        cumulative_reward += sum(rewards.values())
        
        # Placeholder for training step (you need to implement your RL algorithm here)
        # For example, compute loss and update networks
        
        # For simplicity, let's assume we have a target value (dummy value here)
        target = torch.zeros(1)
        loss = 0
        for agent in adversary_agents:
            # Get the predicted value
            actor_input = final_inputs[agent]
            prediction = actor_networks[agent](actor_input)
            # Compute loss (this is a placeholder)
            loss += loss_fn(prediction.unsqueeze(0), target)
        
        # Backpropagation
        gcn_optimizer.zero_grad()
        for optimizer in actor_optimizers.values():
            optimizer.zero_grad()
        
        loss.backward()
        
        gcn_optimizer.step()
        for optimizer in actor_optimizers.values():
            optimizer.step()
        
        # Update observations
        observations = next_observations
        
        # Check if all agents are done
        done = all(terminations.values()) or all(truncations.values())
    
    # Append cumulative reward for the episode
    episode_rewards.append(cumulative_reward)
    
    # Print rewards every 'print_interval' episodes
    if episode % print_interval == 0:
        avg_reward = sum(episode_rewards[-print_interval:]) / print_interval
        print(f"Episode {episode}: Average Reward: {avg_reward}")


/home/yungisimon/anaconda3/envs/harl/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Episode 10: Average Reward: 7.276664328311708
Episode 20: Average Reward: -5.956658382421368
Episode 30: Average Reward: 7.3109924135416025
Episode 40: Average Reward: -7.55640975848721
Episode 50: Average Reward: 12.434964790954348


### Save the Models

In [10]:
# Create a directory to save models
model_dir = 'saved_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)


def save_models(num_class_A, num_class_B, adversary_agents):
    class_a_agents = adversary_agents[:num_class_A]
    class_b_agents = adversary_agents[num_class_A:num_class_A + num_class_B]
    print(f'ctype:{class_a_agents}')
    print(f'ctype:{class_b_agents}')
    for agent in class_a_agents:
        if agent in class_a_agents:
            print(f"agent A:{agent}")
            torch.save(actor_networks[agent].state_dict(), os.path.join(model_dir, f"actor_class_A.pth"))
            break
    for agent in class_b_agents:
        if agent in class_b_agents:
            print(f"agent B:{agent}")
            torch.save(actor_networks[agent].state_dict(), os.path.join(model_dir, f"actor_class_B.pth"))
            break

    #torch.save(class_a_agents[0].state_dict(), os.path.join(model_dir, f"actor_class_A.pth"))
    #torch.save(class_b_agents[0].state_dict(), os.path.join(model_dir, f"actor_class_B.pth"))


    # # Step 1: Add agent class identifier into observation
    # for agent, obs in observations.items():
    #     # If the agent is an adversary (either Class A or Class B)
    #     if agent in class_a_agents or agent in class_b_agents:
    #         agent_class = 0 if agent in class_a_agents else 1  # Class A: 0, Class B: 1
    #         updated_obs = np.concatenate([obs, [agent_class]])  # Add class identifier
    #     else:
    #         updated_obs = obs  # Non-adversary agents keep their observation
    #     updated_observations[agent] = updated_obs

save_models(num_class_a, num_class_b, adversary_agents)

# Save Actor networks
#for i in range(1):
#    torch.save(actor_networks[agent].state_dict(), os.path.join(model_dir, f"actor_{agent}.pth"))

# Save GCN model
torch.save(gcn_layer.state_dict(), os.path.join(model_dir, "gcn_model.pth"))

print("Models saved successfully.")


ctype:['adversary_0', 'adversary_1', 'adversary_2']
ctype:['adversary_3']
agent A:adversary_0
agent B:adversary_3
Models saved successfully.


### Load the Models and Test with Different Number of Adversaries

In [11]:
# Set up the environment with a different number of adversaries
new_num_adversaries = 6  # Change the number of adversaries
env = simple_tag_v2.parallel_env(render_mode=None, num_adversaries=new_num_adversaries, num_good=1, num_obstacles=2)
env.reset()

# Update adversary agents list
adversary_agents = [agent for agent in env.agents if 'adversary' in agent]
good_agents = [agent for agent in env.agents if 'agent' in agent]

# Re-initialize embedding and GCN layers
embedding_layer = LinearEmbedding(input_dim=5, embed_dim=embedding_dim)  # Same as before
gcn_layer = GCNLayer(input_dim=embedding_dim, output_dim=gcn_output_dim)
gcn_layer.load_state_dict(torch.load(os.path.join(model_dir, "gcn_model.pth")))

class_a_agents = ['adversary_0', 'adversary_1', 'adversary_2', 'adversary_3']
class_b_agents = ['adversary_4', 'adversary_5']

# Re-initialize Actor networks for new agents and load the saved models
actor_networks = {}
for agent in adversary_agents:
    print(f"Agent: {agent}")
    input_dim = len(env.observation_space(agent).low) + 1 + gcn_output_dim  # Adjust if observation space changes
    output_dim = env.action_space(agent).n  # Assuming discrete action space
    actor_net = ActorNetwork(input_dim, output_dim)
    # Load the saved model (using the first saved model for simplicity)
    if agent in class_a_agents:
        print(f"Agent: {agent} class A")
        actor_net.load_state_dict(torch.load(os.path.join(model_dir, f"actor_class_A.pth")))
        actor_networks[agent] = actor_net
    if agent in class_b_agents:
        print(f"Agent: {agent} class B")
        actor_net.load_state_dict(torch.load(os.path.join(model_dir, f"actor_class_B.pth")))
        actor_networks[agent] = actor_net

    # Adjust the logic as per your agent classes
    #saved_agent = 'adversary_0' if 'adversary_0' in agent or 'adversary_1' in agent or 'adversary_2' in agent else 'adversary_3'
    #actor_net.load_state_dict(torch.load(os.path.join(model_dir, f"actor_{saved_agent}.pth")))
    #actor_networks[agent] = actor_net

print("Models loaded successfully.")


Agent: adversary_0
Agent: adversary_0 class A


RuntimeError: Error(s) in loading state_dict for ActorNetwork:
	size mismatch for fc1.weight: copying a param with shape torch.Size([64, 35]) from checkpoint, the shape in current model is torch.Size([64, 39]).